In [2]:
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")




    


In [3]:
# 1.Load 导入Document Loaders
def load_documents():
    base_dir = './OneFlower' 
    documents = []
    for file in os.listdir(base_dir): 
        # 构建完整的文件路径    
        file_path = os.path.join(base_dir, file)    
        if file.endswith('.pdf'):        
            loader = PyPDFLoader(file_path)        
            documents.extend(loader.load())    
        elif file.endswith('.docx'):         
            loader = Docx2txtLoader(file_path)        
            documents.extend(loader.load())    
        elif file.endswith('.txt'):       
            loader = TextLoader(file_path)      
            documents.extend(loader.load())
    return documents

In [4]:
documents=load_documents()

In [5]:
# Step 2: Splitting - 分割文本
def split_documents(docs):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # 每个块的大小（字符）
        chunk_overlap=10,  # 块之间的重叠部分
        length_function=len
    )
    return splitter.split_documents(docs)

In [6]:
splits = split_documents(documents)

In [10]:
from langchain_community.vectorstores import Qdrant
from langchain_huggingface import HuggingFaceEmbeddings

os.environ["HF_HOME"] = "/home/fiona/data/hf_cache"
def create_vector_store(splits):
    # 使用自定义DeepSeek嵌入
   # 使用本地嵌入模型
    embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",  # DeepSeek 推荐的中文模型
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )
    
    # 创建向量库
    vectorstore = Qdrant.from_documents(
        documents=splits,
        embedding=embeddings,
        location=":memory:",
        collection_name="flower_knowledge"
    )
    return vectorstore



In [11]:
vectorstore=create_vector_store(splits)

In [16]:
##Step 4 & 5: Retrieval + Output - 检索与生成
def build_qa_chain(vectorstore):
    # 初始化 DeepSeek 模型（假设兼容 OpenAI API）
    llm = ChatOpenAI(
        openai_api_key=api_key,
        openai_api_base="https://api.deepseek.com/v1",
        model_name="deepseek-chat",
        temperature=0.3
    )

    # 定义提示模板
    prompt = ChatPromptTemplate.from_template(
        """你是一名“易速鲜花”客服助手，请严格根据以下上下文回答问题：
        ---
        上下文：{context}
        ---
        问题：{question}
        答案："""
    )

    # 构建检索器
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # 返回最相关的3个块
    # 组合链
    return (
        {"context": retriever, "question": RunnablePassthrough()} 
        | prompt 
        | llm 
    )

In [17]:
qa_chain = build_qa_chain(vectorstore)

tags=['Qdrant', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x7f74d493b750> search_kwargs={'k': 3}


In [18]:
question="我女朋友生日快到了，我想送一个一束花，推荐一下，并且给出理由"
response = qa_chain.invoke(question)
print("回答：", response.content)

回答： 根据您女朋友生日送花的需求，我为您推荐以下三种寓意美好的花束，并附上推荐理由：

1. **绣球花（Hydrangea）花束**
   - **花语**：真诚、感恩、美好祝愿
   - **推荐理由**：绣球花饱满的团簇造型象征圆满幸福，特别适合表达对恋人的真挚情感，蓝色或粉色绣球更能传递浪漫生日祝福。

2. **蝴蝶兰（Phalaenopsis）花盒**
   - **花语**：高贵、优雅、美丽的感动
   - **推荐理由**：兰科花卉自带高级感，花期长达1-2个月，象征永恒的爱意。适合送给气质优雅的女生，搭配珍珠装饰更显珍贵。

3. **铃兰（Lily of the Valley）混搭花束**
   - **花语**：幸福、纯洁、深挚的爱
   - **推荐理由**：铃兰被称为"幸福归来"，小巧铃铛状花朵清新脱俗，搭配白色郁金香或粉玫瑰，能完美表达生日祝福与爱意。

> 小建议：可以附上手写卡片，结合花语表达心意。例如："愿这束铃兰带给你整年的幸福，就像你带给我的每一天"
